In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import ee, eemont
from forestry_carbon_arr.core import ForestryCarbonARR
from forestry_carbon_arr.utils.zarr_utils import save_dataset_efficient_zarr, load_dataset_zarr

import gcsfs
import os

fs = gcsfs.GCSFileSystem(project=os.getenv("GOOGLE_CLOUD_PROJECT"), token='/usr/src/app/user_id.json')


forestry = ForestryCarbonARR(config_path='./00_input/korindo.json')
forestry.initialize_gee()

✓ GEE Initialized successfully
  Credentials Path: /usr/src/app/user_id.json - loaded successfully


In [3]:
# aoi
from forestry_carbon_arr.core.utils import DataUtils
import geopandas as gpd
import geemap

data_utils = DataUtils(forestry.config, use_gee=True)
aoi_gpd, aoi_ee = data_utils.load_geodataframe_gee(forestry.config["AOI_path"])

aoi_gpd_utm = aoi_gpd.to_crs(epsg=32749)

print(f"✅ AOI loaded: {len(aoi_gpd_utm)} features")
print(f"   Area: {aoi_gpd_utm.geometry.area.sum()/10000:.2f} hectares")

/opt/venv/lib/python3.12/site-packages/geemap/conversion.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


✅ AOI loaded: 1 features
   Area: 144217.67 hectares


In [4]:
pl_dir_apr27 = forestry.config.get('pl_dir_apr27')
pl_dir_jun8 = forestry.config.get('pl_dir_jun8')
pl_dir_jul1 = forestry.config.get('pl_dir_jul1')
pl_dir_jul30 = forestry.config.get('pl_dir_jul30')
pl_dir_sep21_2024 = forestry.config.get('pl_dir_sep21_2024')
pl_dir_sep3_2024 = forestry.config.get('pl_dir_sep3_2024')

In [5]:
# pl_dir_jul1

In [18]:
## load the COG and normalize mean, std scale to 0-10000
from gee_lib.osi.spectral_indices.utils import normalization_100

sep3_2024_img = ee.Image.loadGeoTIFF(os.path.join(pl_dir_sep3_2024, "composite_file_format.tif"))
sep_21_2024_img = ee.Image.loadGeoTIFF(os.path.join(pl_dir_sep21_2024, "composite_file_format.tif"))
apr_27_img = ee.Image.loadGeoTIFF(os.path.join(pl_dir_apr27, "composite_file_format.tif"))
jun_8_img = ee.Image.loadGeoTIFF(os.path.join(pl_dir_jun8, "composite_file_format.tif"))
jul_1_img = ee.Image.loadGeoTIFF(os.path.join(pl_dir_jul1, "composite_file_format.tif"))
jul_30_img = ee.Image.loadGeoTIFF(os.path.join(pl_dir_jul30,"composite_file_format.tif"))

sep3_2024_img = normalization_100(sep3_2024_img, pca_scale=3, AOI=aoi_ee.geometry(), scale_factor=10000)
sep_21_2024_img = normalization_100(sep_21_2024_img, pca_scale=3, AOI=aoi_ee.geometry(), scale_factor=10000)
apr_27_img = normalization_100(apr_27_img, pca_scale=3, AOI=aoi_ee.geometry(), scale_factor=10000)
jun_8_img = normalization_100(jun_8_img, pca_scale=3, AOI=aoi_ee.geometry(), scale_factor=10000)
jul_1_img = normalization_100(jul_1_img, pca_scale=3, AOI=aoi_ee.geometry(), scale_factor=10000)
jul_30_img = normalization_100(jul_30_img, pca_scale=3, AOI=aoi_ee.geometry(), scale_factor=10000)

In [19]:
sep3_2024_img.bandNames().getInfo()

['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']

In [20]:
# from wmts_manager import WMTSManager

# wmts = WMTSManager(project_name=forestry.config['project_name'], aoi=aoi_ee.geometry(), clear_cache_first=True)
# wmts.addLayer(jul_30_img, {'bands': ['B5', 'B7', 'B3'],
#    'min': 0,
#    'max': 6000,
#    'gamma': 1.5}, 'jul_30_img'
#              )

# wmts.publish()

In [21]:
## cloud masking start
udm_sep3_2024 = ee.Image.loadGeoTIFF(os.path.join(pl_dir_sep3_2024, "composite_udm2_file_format.tif"))
udm_sep21_2024 = ee.Image.loadGeoTIFF(os.path.join(pl_dir_sep21_2024, "composite_udm2_file_format.tif"))
udm_apr27 =  ee.Image.loadGeoTIFF(os.path.join(pl_dir_apr27, "composite_udm2_file_format.tif"))
udm_jun8 = ee.Image.loadGeoTIFF(os.path.join(pl_dir_jun8, "composite_udm2_file_format.tif"))
udm_jul1 = ee.Image.loadGeoTIFF(os.path.join(pl_dir_jul1, "composite_udm2_file_format.tif"))
udm_jul30 = ee.Image.loadGeoTIFF(os.path.join(pl_dir_jul30, "composite_udm2_file_format.tif"))


In [22]:
udm_apr27.bandNames().getInfo()

['clear',
 'snow',
 'shadow',
 'haze_light',
 'haze_heavy',
 'cloud',
 'confidence',
 'udm1']

In [23]:
udm_jun8.bandNames().getInfo()

['clear',
 'snow',
 'shadow',
 'haze_light',
 'haze_heavy',
 'cloud',
 'confidence',
 'udm1']

In [24]:
### two band to check in UDM
# condition no cloud

confidence_threshold = 90  # >90 is clear
#using other


def cloud_masking(img, udm_img, confidence_threshold):
    """
    Apply cloud masking using UDM with explicit reprojection.
    """
    # Create masks
    clear_val = 1
    no_snow_val = 0
    no_shadow_val = 0
    no_haze_light_val = 0
    no_haze_heavy_val = 0
    no_cloud_val = 0

    clear_mask = udm_img.select('clear').eq(clear_val)
    no_cloud = udm_img.select('snow').eq(no_snow_val)
    no_shadow = udm_img.select('shadow').eq(no_shadow_val)
    no_haze_light = udm_img.select('haze_light').eq(no_haze_light_val)
    no_haze_heavy = udm_img.select('haze_heavy').eq(no_haze_heavy_val)
    no_cloud = udm_img.select('cloud').eq(no_cloud_val)
    confidence_mask = udm_img.select('confidence').gt(confidence_threshold)
    
    # combined_mask = clear_mask.And(confidence_mask)
    combined_mask = clear_mask.And(confidence_mask.And(no_shadow.And(no_haze_light.And(no_haze_heavy.And(no_cloud.And(no_cloud))))))
    
    # Reproject UDM mask to match image's first band projection
    # This is critical for proper alignment
    img_first_band = img.select(0)
    combined_mask = combined_mask.reproject(
        crs=img_first_band.projection().crs(),
        scale=img_first_band.projection().nominalScale()
    )
    
    # Apply mask
    cloudless_img = img.updateMask(combined_mask)
    
    return cloudless_img

sep3_2024_img_cloudless = cloud_masking(sep3_2024_img, udm_sep3_2024, confidence_threshold)
sep_21_2024_img_cloudless = cloud_masking(sep_21_2024_img, udm_sep21_2024, confidence_threshold)
apr_27_img_cloudless = cloud_masking(apr_27_img, udm_apr27, confidence_threshold)
jun_8_img_cloudless = cloud_masking(jun_8_img, udm_jun8, confidence_threshold)
jul_1_img_cloudless = cloud_masking(jul_1_img, udm_jul1, confidence_threshold)
jul_30_img_cloudless = cloud_masking(jul_30_img, udm_jul30, confidence_threshold)

In [25]:
jul_1_img_cloudless.propertyNames().getInfo()

['system:bands', 'system:band_names']

In [26]:
from wmts_manager import WMTSManager

wmts = WMTSManager(project_name=forestry.config['project_name'], aoi=aoi_ee.geometry(), clear_cache_first=True)

wmts.addLayer(sep3_2024_img_cloudless, {'bands': ['B5', 'B7', 'B3'],
   'min': 0,
   'max': 6000,
   'gamma': 1.5}, 'sep3_2024_img_cloudless'
             )

wmts.addLayer(sep_21_2024_img_cloudless, {'bands': ['B5', 'B7', 'B3'],
   'min': 0,
   'max': 6000,
   'gamma': 1.5}, 'sep_21_2024_img_cloudless'
             )

wmts.addLayer(apr_27_img_cloudless, {'bands': ['B5', 'B7', 'B3'],
   'min': 0,
   'max': 6000,
   'gamma': 1.5}, 'apr_27_img_cloudless'
             )

wmts.addLayer(jun_8_img_cloudless, {'bands': ['B5', 'B7', 'B3'],
   'min': 0,
   'max': 6000,
   'gamma': 1.5}, 'jun_8_img_cloudless'
             )

wmts.addLayer(jul_1_img_cloudless, {'bands': ['B5', 'B7', 'B3'],
   'min': 0,
   'max': 6000,
   'gamma': 1.5}, 'jul_1_img_cloudless'
             )

wmts.addLayer(jul_30_img_cloudless, {'bands': ['B5', 'B7', 'B3'],
   'min': 0,
   'max': 6000,
   'gamma': 1.5}, 'jul_30_img_cloudless'
             )

wmts.publish()

INFO:wmts_manager:WMTSManager initialized for project: korindo
INFO:wmts_manager:Added layer: sep3_2024_img_cloudless
INFO:wmts_manager:Added layer: sep_21_2024_img_cloudless
INFO:wmts_manager:Added layer: apr_27_img_cloudless
INFO:wmts_manager:Added layer: jun_8_img_cloudless
INFO:wmts_manager:Added layer: jul_1_img_cloudless
INFO:wmts_manager:Added layer: jul_30_img_cloudless
INFO:wmts_manager:Generating map IDs for 6 layers...


Generating GEE Map IDs...


INFO:wmts_manager:AOI processed: {'minx': 111.706493374869, 'miny': -0.457839253478653, 'maxx': 112.109836017418, 'maxy': -0.167195667545911}
INFO:wmts_manager:Publishing 6 layers to WMTS...
INFO:gee_integration:GEE Integration Manager initialized:
INFO:gee_integration:  FastAPI URL: http://fastapi:8000
INFO:gee_integration:  MapStore Config: /usr/src/app/mapstore/configs/localConfig.json
INFO:gee_integration:Processing GEE analysis: korindo
INFO:gee_integration:🧹 Clearing duplicate projects before processing new analysis...


✅ Centroid calculated successfully with error margin 1
Calculated bbox from coordinates: {'minx': 111.706493374869, 'miny': -0.457839253478653, 'maxx': 112.109836017418, 'maxy': -0.167195667545911}
✅ AOI processed successfully:
   - Center: [111.90816469614359, -0.3125187262245936]
   - Area: Unknown
   - BBox: {'minx': 111.706493374869, 'miny': -0.457839253478653, 'maxx': 112.109836017418, 'maxy': -0.167195667545911}


INFO:cache_manager:No existing catalog entries to check for duplicates
INFO:gee_integration:✅ Duplicate clearing successful: 0 duplicates cleared, 0 unique projects kept
INFO:gee_integration:✅ Cache cleared: 0 duplicate entries, kept 0 unique projects
INFO:gee_integration:Using complex layer info for 'sep3_2024_img_cloudless': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Using complex layer info for 'sep_21_2024_img_cloudless': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Using complex layer info for 'apr_27_img_cloudless': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Using complex layer info for 'jun_8_img_cloudless': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Using complex layer info for 'jul_1_img_cloudless': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Using complex layer info for 'jul_30_img_cloudless': ['tile_url', 'name', 'description', 'vis_params'

{'status': 'success',
 'project_id': 'korindo_20251121_125520',
 'project_name': 'korindo',
 'fastapi_registration': {'status': 'success',
  'message': 'MapStore catalog updated successfully',
  'layers_count': 6},
 'proxy_urls_creation': {'status': 'success',
  'message': 'Created 6 proxy URLs',
  'proxy_urls': {'sep3_2024_img_cloudless': {'proxy_url': 'http://fastapi:8000/tiles/korindo_20251121_125520/sep3_2024_img_cloudless/{z}/{x}/{y}',
    'original_url': 'https://earthengine.googleapis.com/v1/projects/remote-sensing-476412/maps/43c32a2a03963cd9806c51f98684c3e6-790eb3d43935aa11bb0b50c7b6016b61/tiles/{z}/{x}/{y}',
    'layer_name': 'sep3_2024_img_cloudless',
    'description': 'SEP3_2024_IMG_CLOUDLESS visualization from GEE analysis'},
   'sep_21_2024_img_cloudless': {'proxy_url': 'http://fastapi:8000/tiles/korindo_20251121_125520/sep_21_2024_img_cloudless/{z}/{x}/{y}',
    'original_url': 'https://earthengine.googleapis.com/v1/projects/remote-sensing-476412/maps/f2adfe6ce200a3da26

In [27]:
### based on the images converted to the gee object, we will merge them into gee image collection 
# Function to standardize images for homogeneous collection
def standardize_image(img):
    """Ensure all images have consistent type and value range"""
    # Convert to float and clamp to 0-10000 range
    standardized = img.toFloat().clamp(0, 10000)
    return standardized

# Standardize all images before creating collection
sep3_2024_img_cloudless = standardize_image(sep3_2024_img_cloudless)
sep_21_2024_img_cloudless = standardize_image(sep_21_2024_img_cloudless)
apr_27_img_cloudless = standardize_image(apr_27_img_cloudless)
jun_8_img_cloudless = standardize_image(jun_8_img_cloudless)
jul_1_img_cloudless = standardize_image(jul_1_img_cloudless)
jul_30_img_cloudless = standardize_image(jul_30_img_cloudless)

sep3_2024_img_cloudless = sep3_2024_img_cloudless.set('system:time_start', ee.Date.fromYMD(2024, 9, 3).millis())
sep_21_2024_img_cloudless = sep_21_2024_img_cloudless.set('system:time_start', ee.Date.fromYMD(2024, 9, 21).millis())
apr_27_img_cloudless = apr_27_img_cloudless.set('system:time_start', ee.Date.fromYMD(2025, 4, 27).millis())
jun_8_img_cloudless = jun_8_img_cloudless.set('system:time_start', ee.Date.fromYMD(2025, 6, 8).millis())
jul_1_img_cloudless= jul_1_img_cloudless.set('system:time_start', ee.Date.fromYMD(2025, 7, 1).millis())
jul_30_img_cloudless = jul_30_img_cloudless.set('system:time_start', ee.Date.fromYMD(2025, 7, 30).millis())

list_ee_image = [sep_21_2024_img_cloudless, apr_27_img_cloudless, jun_8_img_cloudless, jul_1_img_cloudless, jul_30_img_cloudless]
planet_scope_col = ee.ImageCollection(list_ee_image)

list_ee_image = [sep_21_2024_img_cloudless, apr_27_img_cloudless, jun_8_img_cloudless, jul_1_img_cloudless, jul_30_img_cloudless]
planet_scope_col= ee.ImageCollection(list_ee_image)

In [28]:
planet_scope_col.first().get('system:band_names').getInfo()

['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']

In [29]:
planet_scope_col.first().get('system:time_start')

In [30]:
# from forestry_carbon_arr.utils import fill_temporal_gaps_linear

# # Use on your SG-smoothed collection
# bands_to_fill = ['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']  # or the bands you smoothed
# planet_scope_col_interpolated = fill_temporal_gaps_linear(planet_scope_col, bands_to_fill)

# print('Original monthly (with NaNs):', planet_scope_col.size().getInfo())
# print('Filled monthly collection:', planet_scope_col_interpolated.size().getInfo())

# recent_img = planet_scope_col_interpolated.filter(
#     ee.Filter.eq('system:time_start', ee.Date.fromYMD(2025, 7, 30).millis())
# ).first()

# wmts.addLayer(recent_img, {'bands': ['B5', 'B7', 'B3'],
#    'min': 0,
#    'max': 6000,
#    'gamma': 1.5}, 'recent_img_FIX'
#              )

# wmts.publish()

In [34]:
# # use native function mosaic instead in GEE
# def mosaic_with_explicit_priority(image_list, priority_order='time'):
#     """
#     Mosaic with explicit priority control.
    
#     Parameters:
#     -----------
#     image_list : list of ee.Image
#         List of images (most recent should be last)
#     priority_order : str
#         'time' = sort by timestamp, 'manual' = use list order
#     """
#     if priority_order == 'time':
#         # Sort by timestamp (most recent last)
#         collection = ee.ImageCollection(image_list).sort('system:time_start', False)
#     else:
#         # Use list order (last image = highest priority)
#         collection = ee.ImageCollection(image_list)
    
#     # Mosaic: last valid pixel wins
#     mosaicked = collection.mosaic()
    
#     return mosaicked
# mosaick only return the last image for some reason :(

# Use it
# mosaicked_image = mosaic_with_explicit_priority(planet_scope_col, priority_order='time')
mosaicked_image = planet_scope_col.median()


In [35]:
mosaicked_image.bandNames().getInfo()

['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']

In [36]:
wmts.addLayer(mosaicked_image, {'bands': ['B5', 'B7', 'B3'],
   'min': 0,
   'max': 6000,
   'gamma': 1.5}, 'mosaicked_image_median'
             )

wmts.publish()

INFO:wmts_manager:Added layer: mosaicked_image_median
INFO:wmts_manager:Generating map IDs for 8 layers...


Generating GEE Map IDs...


INFO:wmts_manager:Publishing 8 layers to WMTS...
INFO:gee_integration:GEE Integration Manager initialized:
INFO:gee_integration:  FastAPI URL: http://fastapi:8000
INFO:gee_integration:  MapStore Config: /usr/src/app/mapstore/configs/localConfig.json
INFO:gee_integration:Processing GEE analysis: korindo
INFO:gee_integration:🧹 Clearing duplicate projects before processing new analysis...
INFO:cache_manager:No existing catalog entries to check for duplicates
INFO:gee_integration:✅ Duplicate clearing successful: 0 duplicates cleared, 0 unique projects kept
INFO:gee_integration:✅ Cache cleared: 0 duplicate entries, kept 0 unique projects
INFO:gee_integration:Using complex layer info for 'sep3_2024_img_cloudless': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Using complex layer info for 'sep_21_2024_img_cloudless': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Using complex layer info for 'apr_27_img_cloudless': ['tile_url', 'name', 'descrip

{'status': 'success',
 'project_id': 'korindo_20251121_130255',
 'project_name': 'korindo',
 'fastapi_registration': {'status': 'success',
  'message': 'MapStore catalog updated successfully',
  'layers_count': 8},
 'proxy_urls_creation': {'status': 'success',
  'message': 'Created 8 proxy URLs',
  'proxy_urls': {'sep3_2024_img_cloudless': {'proxy_url': 'http://fastapi:8000/tiles/korindo_20251121_130255/sep3_2024_img_cloudless/{z}/{x}/{y}',
    'original_url': 'https://earthengine.googleapis.com/v1/projects/remote-sensing-476412/maps/43c32a2a03963cd9806c51f98684c3e6-cd260941a3d159b96363e52c3c5cc46b/tiles/{z}/{x}/{y}',
    'layer_name': 'sep3_2024_img_cloudless',
    'description': 'SEP3_2024_IMG_CLOUDLESS visualization from GEE analysis'},
   'sep_21_2024_img_cloudless': {'proxy_url': 'http://fastapi:8000/tiles/korindo_20251121_130255/sep_21_2024_img_cloudless/{z}/{x}/{y}',
    'original_url': 'https://earthengine.googleapis.com/v1/projects/remote-sensing-476412/maps/f2adfe6ce200a3da26

In [ ]:
import ee

# Example: Export single image to GCS
def export_image_to_gcs(image, gcs_bucket, gcs_path, scale=10, crs='EPSG:4326', region=None, max_pixels=1e13):
    """
    Export Earth Engine Image to Google Cloud Storage
    
    Parameters:
    -----------
    image : ee.Image
        Earth Engine Image to export
    gcs_bucket : str
        GCS bucket name (e.g., 'my-bucket' or 'gs://my-bucket')
    gcs_path : str
        Path within bucket (e.g., 'exports/fcd_2020.tif')
    scale : float
        Pixel scale in meters (default: 30)
    crs : str
        Coordinate reference system (default: 'EPSG:4326')
    region : ee.Geometry, optional
        Region to export (default: None, uses image bounds)
    max_pixels : int
        Maximum pixels to export (default: 1e9)
    
    Returns:
    --------
    ee.batch.Task : Export task
    """
    # Clean bucket name (remove gs:// if present)
    if gcs_bucket.startswith('gs://'):
        gcs_bucket = gcs_bucket.replace('gs://', '').split('/')[0]
    
    # Full GCS path
    gcs_uri = f"gs://{gcs_bucket}/{gcs_path}"
    
    # Export parameters
    export_params = {
        'image': image,
        'description': gcs_path.split('/')[-1].replace('.tif', ''),  # Task name
        'bucket': gcs_bucket,
        'fileNamePrefix': gcs_path.replace('.tif', ''),  # Path without extension
        'scale': scale,
        'crs': crs,
        'maxPixels': max_pixels,
        'fileFormat': 'GeoTIFF',
        'formatOptions': {
            'cloudOptimized': True  # COG format
        }
    }
    
    # Add region if provided
    if region is not None:
        export_params['region'] = region
    
    # Create export task
    task = ee.batch.Export.image.toCloudStorage(**export_params)
    
    # Start the task
    task.start()
    
    print(f"✅ Export task started: {gcs_uri}")
    print(f"   Task ID: {task.id}")
    
    return task

In [ ]:
task = export_image_to_gcs(
        image=mosaicked_image.toFloat(),
        gcs_bucket='remote_sensing_saas',
        gcs_path=f'01-korindo/planet_merged/planet_recent_2025_07_mosaicked_image_2024_9_2025_7.tif',
        scale=3,
        crs='EPSG:32749',  # UTM zone for your AOI
        region=aoi_ee.geometry()  # Optional: clip to AOI
    )

In [ ]:
### RUN DIRECTLY TO THIS AFTER STEP IMPORT FORESTRY DONE AND AOI
## case where the tif is too big and splitted by GEE
# Find all tiles matching the prefix
pattern = f"gs://remote_sensing_saas/01-korindo/planet_merged/planet_recent_2025_07_mosaicked_image_2024_9_2025_7*.tif"
tiles = sorted(fs.glob(pattern))

if not tiles:
    print(f"❌ No tiles found matching: {pattern}")

print(f"✅ Found {len(tiles)} tiles to merge:")
for i, tile in enumerate(tiles, 1):
    print(f"   {i}. {tile}")

# Load all tiles as ee.Images
print(f"\n📥 Loading tiles into GEE...")
tile_images = []
for tile in tiles:
    gcs_path = f"gs://{tile}"
    img = ee.Image.loadGeoTIFF(gcs_path)
    tile_images.append(img)
    print(f"   ✓ Loaded: {os.path.basename(tile)}")

# Create ImageCollection and mosaic
print(f"\n🔄 Merging {len(tile_images)} tiles...")
tile_collection = ee.ImageCollection(tile_images)
input_image = tile_collection.mosaic()

# ## after exported -->
# input_image = ee.Image.loadGeoTIFF(os.path.join(f'gs://remote_sensing_saas',f'01-korindo/planet_merged/planet_recent_2025_07_mosaicked_image_2024_9_2025_7.tif'))
input_image.propertyNames().getInfo()    

In [ ]:
input_image.bandNames().getInfo()

In [ ]:
input_image_fix = input_image.select(['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']). \
                                rename(['coastal_blue','blue','green1','green','yellow','red','redE1','nir'])

# Divide all bands by 10000
input_image_fix = input_image_fix.toFloat().divide(10000)

input_image_fix.bandNames().getInfo()

In [ ]:
# forestry.config

In [ ]:
from wfs_manager import WFSManager

AOI = ee.FeatureCollection('projects/remote-sensing-476412/assets/korindo_with_buffer')

wfs = WFSManager(fastapi_url="http://fastapi:8000", wfs_base_url="http://localhost:8001")
wfs.addLayer(AOI, "AOI Boundary")
wfs.publish()

In [ ]:
from wmts_manager import WMTSManager

wmts.addLayer(input_image_fix, {'bands': ['red', 'nir', 'green'],
   'min': 0,
   'max': 0.6,
   'gamma': 1.5}, 'mosaicked_image_fix'
             )

wmts.publish()

In [ ]:
## See imbalance training samples
# Add this diagnostic to your workflow
def analyze_training_data(training_points, label_column='type'):
    """Analyze training data quality"""
    import ee
    
    # Get class distribution
    class_counts = training_points.aggregate_histogram(label_column).getInfo()
    total_samples = training_points.size().getInfo()
    
    print("=" * 60)
    print("TRAINING DATA ANALYSIS")
    print("=" * 60)
    print(f"Total samples: {total_samples}")
    print(f"\nSamples per class:")
    for class_id, count in sorted(class_counts.items()):
        percentage = (count / total_samples) * 100
        status = "✓" if count >= 200 else "⚠" if count >= 50 else "✗"
        print(f"  {status} Class {class_id}: {count} samples ({percentage:.1f}%)")
    
    # Identify issues
    min_samples = min(class_counts.values())
    max_samples = max(class_counts.values())
    imbalance_ratio = max_samples / min_samples if min_samples > 0 else float('inf')
    
    print(f"\nClass imbalance ratio: {imbalance_ratio:.2f}")
    if imbalance_ratio > 10:
        print("⚠ WARNING: Severe class imbalance detected!")
    
    if min_samples < 50:
        print("⚠ WARNING: Some classes have < 50 samples (minimum for good accuracy)")
    if min_samples < 200:
        print("⚠ WARNING: Some classes have < 200 samples (recommended for >80% accuracy)")
    
    return class_counts

In [ ]:
# Set I_satellite to 'Custom' in your config
forestry.config['I_satellite'] = 'Custom'

# Run eligibility with your custom image
el = forestry.run_eligibility(
    config=forestry.config,
    use_gee=True,
    custom_image=input_image_fix  # Your pre-processed image
)

In [ ]:
# Access the LandcoverML instance from eligibility results
lc = el['intermediate_results']['landcover_ml_instance']

## check imbalance
training_points_ee = lc.input_training
analyze_training_data(training_points_ee, label_column='type')

In [ ]:
gbm = el['intermediate_results']['classifier_results']['classified_image_gbm']
gbm.propertyNames().getInfo()

In [ ]:
# Assess accuracy for all algorithms
accuracy_results = forestry.assess_accuracy(
    eligibility_results=el,
    output_dir='./01_output'
)

# # Or assess only specific algorithms
# accuracy_results = forestry.assess_accuracy(
#     eligibility_results=el,
#     algorithms=['fcd', 'gbm'],
#     output_dir='./01_output'
# )

# # Or use custom validation points
# accuracy_results = forestry.assess_accuracy(
#     eligibility_results=el,
#     validation_points=my_validation_points,
#     output_dir='./outputs',
#     label_column='type'  # Use your custom label column
# )

In [ ]:
# from wmts_manager import WMTSManager

# wmts = WMTSManager(project_name=forestry.config['project_name'], aoi=aoi_ee.geometry(), clear_cache_first=True)

## LC
rf = el['intermediate_results']['classifier_results']['classified_image_rf']
# svm = el['intermediate_results']['classifier_results']['classified_image_svm']
# gbm = el['intermediate_results']['classifier_results']['classified_image_gbm']
# cart = el['intermediate_results']['classifier_results']['classified_image_cart']

rf_layer_name = el['intermediate_results']['layer_names']['land_cover_rf']
# svm_layer_name = el['intermediate_results']['layer_names']['land_cover_svm']
# gbm_layer_name = el['intermediate_results']['layer_names']['land_cover_gbm']
# cart_layer_name = el['intermediate_results']['layer_names']['land_cover_cart']

vis_param_lc = el['visualization_params']['land_cover']

wmts.addLayer(rf, vis_param_lc, rf_layer_name)
# wmts.addLayer(svm, vis_param_lc, svm_layer_name)
# wmts.addLayer(gbm, vis_param_lc, gbm_layer_name)
# wmts.addLayer(cart, vis_param_lc, cart_layer_name)
wmts.publish()

In [ ]:
### using the GEE only, is only gets to 73% max. in parallel -->
## Training data should be improved e.g create area polygon for the Forest only,
##### Integrate to python ML project, 
## ds for pixel after scaled with indices (rbgnir si etc) without normalization
## ds normalized result (all bands)
## ds normalized result with the SNIC cluster based (OBIA)
## prioritize to create new OBIA in GEE based on arcgis

In [ ]:
input_image_fix.bandNames().getInfo()

In [ ]:
hist = input_image_fix.select(input_image_fix.bandNames().getInfo()).reduceRegion(
    reducer=ee.Reducer.histogram(maxBuckets=100),
    geometry=aoi_ee.geometry(),
    scale=3,
    maxPixels=1e9,
    bestEffort=True
)

# for band, item in hist.getInfo().items():
#     print(band)
#     print(item)
# bucketMeans tells you the data stat, histogram is count number

In [ ]:
### lets combine with the normalized data and FCD normalized
image_norm_with_fcd = el['intermediate_results']['image_norm_with_spectral_indices_FCD']
image_norm_with_fcd.bandNames().getInfo()

In [ ]:
# map the same name as _normalized
origin_bands = input_image_fix.bandNames().getInfo()
target_bands = image_norm_with_fcd.bandNames().getInfo()

target_rename_bands = []
for band in target_bands:
    if band in origin_bands:
        band = band+'_norm'
    else:
        band
    target_rename_bands.append(band)

image_norm_with_fcd_renamed = image_norm_with_fcd.rename(target_rename_bands)
image_norm_with_fcd_renamed.bandNames().getInfo()

In [ ]:
img_to_export = input_image_fix.addBands(image_norm_with_fcd_renamed)
img_to_export.bandNames().getInfo()

In [ ]:
import ee_extra.Spectral.core as spec_core

# Get all available indices dynamically
indices_dict = spec_core.indices(online=False)
spectral_indices_awesome_list = sorted(list(indices_dict.keys()))

print(f"✅ Loaded {len(spectral_indices_awesome_list)} spectral indices from eemont-osi")
print(f"\nFirst 10 indices: {spectral_indices_awesome_list[:10]}")
print(f"\nLast 10 indices: {spectral_indices_awesome_list[-10:]}")
print(f"\n✅ List ready to use: spectral_indices_awesome_list")

In [ ]:
spectral_indices_awesome_list

In [ ]:
top_20_indices = [
    # Vegetation (6)
    'NDVI',      # 1. Baseline vegetation
    'EVI',       # 2. Enhanced vegetation
    'SAVI',      # 3. Soil-adjusted
    'MSAVI2',    # 4. Modified SAVI (you have)
    'GNDVI',     # 5. Green-based
    'ARVI',      # 6. Atmospherically resistant
    
    # Water & Moisture (4)
    'NDWI',      # 7. Water detection (you have)
    'MNDWI',     # 8. Modified water
    'NDMI',      # 9. Moisture content
    'LSWI',      # 10. Land surface water
    
    # Soil & Urban (3)
    'BSI',       # 11. Bare soil (you have)
    'NDBI',      # 12. Built-up areas
    'UI',        # 13. Urban index
    
    # Stress & Health (3)
    'NBR',       # 14. Burn/stress
    'NDREI',     # 15. Red edge (if available)
    'CRI700',    # 16. Chlorophyll (if red edge available)
    
    # Specialized (4)
    'MTVI2',     # 17. Triangular (you have)
    'VARI',      # 18. Visible (you have)
    'BAI',       # 19. Burn area
    'GEMI'       # 20. Global monitoring
]

In [ ]:
# High priority (add these)
priority_indices = [
    'EVI',      # Enhanced vegetation (better than NDVI for dense vegetation)
    'SAVI',     # Soil-adjusted (important for sparse vegetation)
    'OSAVI',    # Optimized SAVI
    'GNDVI',    # Green-based vegetation
    'MSAVI',    # Modified SAVI (you have MSAVI2, this is different)
    'RVI',      # Ratio vegetation
    'RDVI',     # Renormalized difference
    'ExG',      # Excess green (good for vegetation detection)
    'ExGR',     # Excess green-red (vegetation vs soil)
    'GLI',      # Green leaf index
    'NGRDI',    # Normalized green-red difference
    'TVI',      # Triangular vegetation
    'MSR',      # Modified simple ratio
    'NLI',      # Non-linear index
    'WDRVI'     # Wide dynamic range (less saturation)
]

In [ ]:
# list(sorted(set(['EVI', 'GNDVI', 'SAVI','NBR','WDRVI','WI2','ExG','VIG','ARVI','MNDWI',
#            'NDMI', 'LSWI','NDBI','UI','NDREI','CRI700','BAI','GEMI','OSAVI','RVI','RDVI','ExGR',
#           'GLI', 'NGRDI','TVI','MSR','NLI'])))

In [ ]:
img_to_export.bandNames().getInfo()

In [ ]:
# Short name (recommended)
img_with_indices = img_to_export.spectra_indices(
    index=[
        'ARVI', 'BAI', 'EVI', 'ExG', 'ExGR', 'CRI700',
        'GEMI', 'GLI', 'GNDVI', 'MSR',  'NGRDI', 'NDREI',
        'NLI', 'OSAVI', 'RDVI', 'RDVI', 'SAVI', 'TVI',
        'VIG', 'WDRVI','AVI','SI'
    ],
    G=2.5,
    C1=6.0,
    C2=7.5,
    L=1.0,
    drop=False
)

In [ ]:
img_with_indices.bandNames().getInfo()

In [ ]:
export_image_to_gcs(img_with_indices.toFloat(), gcs_bucket = 'remote_sensing_saas',
                    gcs_path = '01-korindo/planet_merged/planet_with_indices_recent_2025.tif', scale=5, crs='EPSG:32749', region=None, max_pixels=1e13)

In [ ]:
### look for the availability of the spectral indices in the eemont
# Dynamically get all available spectral indices from eemont-osi
# import sys
# sys.path.insert(0, '/usr/src/app/eemont-osi')
import ee_extra.Spectral.core as spec_core

# Get all available indices dynamically
indices_dict = spec_core.indices(online=False)
spectral_indices_awesome_list = sorted(list(indices_dict.keys()))

print(f"✅ Loaded {len(spectral_indices_awesome_list)} spectral indices from eemont-osi")
print(f"\nFirst 10 indices: {spectral_indices_awesome_list[:10]}")
print(f"\nLast 10 indices: {spectral_indices_awesome_list[-10:]}")
print(f"\n✅ List ready to use: spectral_indices_awesome_list")

# Function to get formula and metadata for any spectral index
def get_index_info(index_name):
    """
    Get formula and metadata for a spectral index.
    
    Parameters:
    -----------
    index_name : str
        Name of the spectral index (e.g., 'NDVI', 'EVI', 'SAVI')
    
    Returns:
    --------
    dict : Dictionary containing:
        - formula: Mathematical formula string
        - long_name: Full name of the index
        - bands: List of bands used (N=NIR, R=Red, G=Green, B=Blue, S1=SWIR1, S2=SWIR2, RE1-4=Red Edge)
        - application_domain: Category (vegetation, water, burn, etc.)
        - platforms: Supported satellite platforms
        - reference: Reference URL or DOI
    """
    index_name_upper = index_name.upper()
    
    if index_name_upper not in indices_dict:
        available = [idx for idx in spectral_indices_awesome_list if index_name.upper() in idx.upper()]
        raise ValueError(
            f"Index '{index_name}' not found. "
            f"Did you mean: {available[:5] if available else 'None'}?"
        )
    
    info = indices_dict[index_name_upper].copy()
    return info

def formula(index_name):
    """
    Get the formula for a spectral index.
    
    Parameters:
    -----------
    index_name : str
        Name of the spectral index (e.g., 'NDVI', 'EVI', 'SAVI')
    
    Returns:
    --------
    str : Mathematical formula string using band abbreviations
         Band abbreviations: N (NIR), R (Red), G (Green), B (Blue), 
         S1 (SWIR1), S2 (SWIR2), RE1-4 (Red Edge 1-4)
    
    Examples:
    --------
    >>> formula('NDVI')
    '(N - R)/(N + R)'
    
    >>> formula('EVI')
    'G * ((N - R) / (N + C1 * R - C2 * B + L))'
    """
    info = get_index_info(index_name)
    return info['formula']

# Function to convert formula band abbreviations to OSI band names
def formula_to_osi_bands(formula_str):
    """
    Convert eemont-osi formula band abbreviations to OSI band names.
    
    Mapping:
    - N (NIR) -> nir
    - R (Red) -> red
    - G (Green) -> green
    - B (Blue) -> blue
    - S1 (SWIR1) -> swir1
    - S2 (SWIR2) -> swir2
    - RE1 (Red Edge 1) -> redE1
    - RE2 (Red Edge 2) -> redE2
    - RE3 (Red Edge 3) -> redE3
    - RE4 (Red Edge 4) -> redE4
    - Variables (g, C1, C2, L, etc.) remain as-is
    
    Parameters:
    -----------
    formula_str : str
        Formula string from eemont-osi (e.g., "(N - R)/(N + R)")
    
    Returns:
    --------
    str : Formula with OSI band names (e.g., "(nir - red)/(nir + red)")
    """
    import re
    
    # Mapping from eemont-osi abbreviations to OSI band names
    band_mapping = {
        'N': 'nir',      # Near Infrared
        'R': 'red',      # Red
        'G': 'green',    # Green
        'B': 'blue',     # Blue
        'S1': 'swir1',   # Shortwave Infrared 1
        'S2': 'swir2',   # Shortwave Infrared 2
        'RE1': 'redE1',  # Red Edge 1
        'RE2': 'redE2',  # Red Edge 2
        'RE3': 'redE3',  # Red Edge 3
        'RE4': 'redE4',  # Red Edge 4
    }
    
    # Sort by length (longest first) to avoid partial matches (e.g., RE1 before R)
    sorted_bands = sorted(band_mapping.keys(), key=len, reverse=True)
    
    result = formula_str
    
    # Replace band abbreviations with OSI names
    # Use word boundaries to avoid replacing partial matches in variables
    for abbrev in sorted_bands:
        osi_name = band_mapping[abbrev]
        # Use regex to match whole words only (not part of other words)
        # Pattern: \b matches word boundary, but we need to handle cases like "RE1" in "RE1*RE2"
        pattern = r'\b' + re.escape(abbrev) + r'\b'
        result = re.sub(pattern, osi_name, result)
    
    return result

def formula_osi(index_name):
    """
    Get the formula for a spectral index with OSI band names.
    
    Parameters:
    -----------
    index_name : str
        Name of the spectral index (e.g., 'NDVI', 'EVI', 'NBR')
    
    Returns:
    --------
    str : Mathematical formula string using OSI band names
         (nir, red, green, blue, swir1, swir2, redE1-4)
    
    Examples:
    --------
    >>> formula_osi('NDVI')
    '(nir - red)/(nir + red)'
    
    >>> formula_osi('NBR')
    '(nir - swir2)/(nir + swir2)'
    """
    formula_orig = formula(index_name)
    return formula_to_osi_bands(formula_orig)

# Example usage
print("\n" + "="*60)
print("Example: Getting formula for NDVI and NBR")
print("="*60)
try:
    # NDVI example
    ndvi_formula = formula('NDVI')
    ndvi_formula_osi = formula_osi('NDVI')
    ndvi_info = get_index_info('NDVI')
    print(f"\nNDVI:")
    print(f"  Original Formula: {ndvi_formula}")
    print(f"  OSI Band Names: {ndvi_formula_osi}")
    print(f"  Long Name: {ndvi_info['long_name']}")
    print(f"  Bands: {ndvi_info['bands']}")
    
    # NBR example (shows S2 -> swir2)
    nbr_formula = formula('NBR')
    nbr_formula_osi = formula_osi('NBR')
    nbr_info = get_index_info('NBR')
    print(f"\nNBR (Normalized Burn Ratio):")
    print(f"  Original Formula: {nbr_formula}")
    print(f"  OSI Band Names: {nbr_formula_osi}")
    print(f"  Long Name: {nbr_info['long_name']}")
    print(f"  Bands: {nbr_info['bands']} (S2 = SWIR2)")
    print(f"  Domain: {nbr_info['application_domain']}")
    
except Exception as e:
    print(f"Error: {e}")

